In [1]:
!pip install -r requirements.txt

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Ignoring openjtalk: markers 'sys_platform != "linux"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.5/751.5 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 kB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 4.2 MB/s eta 0:0

In [18]:
import os
import shutil
from git.repo import Repo
from IPython.display import Audio

In [4]:
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print('create a new folder at path ' + path)
    else:
        print('folder at path ' + path + ' has been created')

def create_file(path):
    file = os.path.exists(path)
    if not file:
          with open (path, 'w'):
            print('create a new file at path ' + path)
    else:
          print('file at path ' + path + ' has been created')

In [5]:
# 创建config.yml
dataset_path = "./Data"
config_yml_path = "./Data/config.yml"
try:
    mkdir(dataset_path)
    from config import Config
    config = Config(config_yml_path)
except SystemExit as msg:
    print(msg)

create a new folder at path ./Data
已根据默认配置文件default_config.yml生成配置文件./Data/config.yml。请按该配置文件的说明进行配置后重新运行。
如无特殊需求，请勿修改default_config.yml或备份该文件。
0


In [12]:
# 初始化项目
from config import config
mkdir(config.resample_config.in_dir)
mkdir(config.resample_config.out_dir)
mkdir(config.dataset_path + "filelists")
create_file(config.preprocess_text_config.transcription_path)
create_file(config.preprocess_text_config.train_path)
create_file(config.preprocess_text_config.val_path)
shutil.copy('./configs/config.json', config.dataset_path)
# TODO: 原始语音文件导入到 './Data/audios/raw'
# TODO: 语音文本标注写入到 './Data/filelists/transcripition.list'
print('init project successfully')

folder at path Data/audios/raw has been created
folder at path Data/audios/resampled has been created
folder at path Data/filelists has been created
file at path Data/filelists/transcription.list has been created
file at path Data/filelists/train.list has been created
file at path Data/filelists/val.list has been created
init project successfully


In [20]:
# 下载bert模型
if not os.path.exists('./bert/chinese-roberta-wwm-ext-large/pytorch_model.bin'):
    shutil.rmtree('./bert/chinese-roberta-wwm-ext-large')
    Repo.clone_from('https://huggingface.co/hfl/chinese-roberta-wwm-ext-large.git',
                    to_path='./bert/chinese-roberta-wwm-ext-large',
                    branch='main')

if not os.path.exists('./bert/deberta-v2-large-japanese-char-wwm/pytorch_model.bin'):
    shutil.rmtree('./bert/deberta-v2-large-japanese-char-wwm')
    Repo.clone_from('https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm',
                    to_path='./bert/deberta-v2-large-japanese-char-wwm',
                    branch='main')

if not os.path.exists('./bert/deberta-v3-large/pytorch_model.bin'):
    shutil.rmtree('./bert/deberta-v3-large')
    Repo.clone_from('https://huggingface.co/microsoft/deberta-v3-large.git',
                    to_path='./bert/deberta-v3-large',
                    branch='main')

In [24]:
# 下载emotional模型
if not os.path.exists('./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/pytorch_model.bin'):
    shutil.rmtree('./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim')
    Repo.clone_from('https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim.git',
                    to_path='./emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim', 
                    branch='main')

In [23]:
# 试听导入音频
audio_path = './Data/audios/raw/depart.wav'
Audio(audio_path, rate=44100)

In [25]:
# 音频数据重采样
!python ./resample.py

35it [00:00, 91.68it/s]
音频重采样完毕!


In [26]:
# 试听重采样导出音频
audio_path = './Data/audios/resampled/depart.wav'
Audio(audio_path, rate=44100)

In [28]:
# 转录语音文本转为拼音
!python ./preprocess_text.py

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
  0%|                                                    | 0/35 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.896 seconds.
Prefix dict has been built successfully.
100%|███████████████████████████████████████████| 35/35 [00:01<00:00, 30.60it/s]
总重复音频数：0，总未找到的音频数:0
训练集和验证集生成完成！


In [29]:
# 生成bert文件
!python ./bert_gen.py

  0%|                                                    | 0/35 [00:00<?, ?it/s]Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./bert/chinese-roberta-wwm-ext-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'c

In [30]:
# 生成emo文件
!python ./emo_gen.py

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
  0%|                                                    | 0/35 [00:00<?, ?it/s]2023-12-10 16:51:35.849893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 16:51:35.849893: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 16:51:37.378038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [33]:
# 执行训练
!python ./train_ms.py

2023-12-10 16:57:55.406585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 16:57:56.400354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
加载config中的配置localhost
加载config中的配置10086
加载config中的配置1
加载config中的配置0
加载config中的配置0
加载环境变量 
MASTER_ADDR: localhost,
MASTER_PORT: 10086,
WORLD_SIZE: 1,
RANK: 0,
LOCAL_RANK: 0
12-10 16:58:00 INFO     | data_utils.py:62 | Init dataset...
100%|████████████████████████████████████████| 31/31 [00:00<00:00, 20917.54it/s]
12-10 16:58:00 INFO     | data_utils.py:77 | skipped: 1, total: 31
12-10 16:58:00 INFO     | data_utils.py:62 | Init dataset...
100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 13684.52it/s]
12-10 16:58:00 INFO     | data_ut

In [68]:
# 推理
!python terminal_infer.py -m "./Data/models/G_2000.pth" -c "./Data/config.json" --text="你好，世界" --speaker='virtuosa' 

| torch.distributed.nn.jit.instantiator | INFO | Created a temporary directory at /tmp/tmpii5ysdkb
| torch.distributed.nn.jit.instantiator | INFO | Writing /tmp/tmpii5ysdkb/_remote_module_non_scriptable.py
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
| __main__ | INFO | Enable DEBUG-LEVEL log
Data/models/G_2000.pth
| utils | INFO | Loaded checkpoint 'Data/models/G_2000.pth' (iteration 286)
运行参数：Namespace(config='./Data/config.json', debug=False, length_scale=1, model='./Data/models/G_2000.pth', noise=0.6, noise_w=0.8, sdp=0.2, share=False, speaker='virtuosa', text='你好，世界')
你好，世界 virtuosa 0.2 0.6 0.8 1 ZH None 0
Building prefix dict from the default dictionary ...
| jieba | DEBUG | Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
| jieba | DEBUG | Loading model from cache /tmp/jieba.cache
Loading model cost 0.840 seconds.
| jieba | 